In [52]:
def string_to_geoid(input_string, state_fips_mapping, county_fips_mapping):
    """
    Convert a descriptive string to a GEOID.

    Parameters:
    input_string (str): A string in the format "Block Group X; Census Tract YYY.YY; County Name; State Name".
    state_fips_mapping (dict): A dictionary mapping state names to their FIPS codes.
    county_fips_mapping (dict): A dictionary mapping county names and state names to their FIPS codes.

    Returns:
    str: A 12-digit GEOID.
    """
    # Split the input string and extract relevant parts
    parts = input_string.split('; ')
    block_group = parts[0].split(' ')[2]
    census_tract = parts[1].split(' ')[2]
    county_name = parts[2]
    state_name = parts[3]

    # Get the state FIPS code
    state_fips = state_fips_mapping.get(state_name)

    # Get the county FIPS code
    county_fips = county_fips_mapping.get((county_name, state_name))

    # Format the census tract as a 6-digit number
    tract_fips = f"{int(census_tract.split('.')[0]):04d}{int(census_tract.split('.')[1]):02d}"

    # Concatenate the parts to form the GEOID
    geoid = f"{state_fips}{county_fips}{tract_fips}{block_group}"

    return geoid

# Example usage
input_string = "Block Group 1; Census Tract 103.04; Hamilton County; Tennessee"
state_fips_mapping = {"Tennessee": "47"}
county_fips_mapping = {("Hamilton County", "Tennessee"): "065"}

geoid = string_to_geoid(input_string, state_fips_mapping, county_fips_mapping)
print("GEOID:", geoid)


GEOID: 470650103041


In [98]:
import pandas as pd
pd.set_option('display.max_columns', None)

income  = pd.read_csv('/home/rishav/Programs/routing/data/ACSDT5Y2022.B19013-Data.csv')[1:]
income['GEOID'] = income['GEO_ID'].apply(lambda x: x.split('US')[1] if len(x.split('US')) > 1 else None)
income = income.rename({'B19013_001E': 'median_income', 'B19013_001M': 'margin'}, axis=1)
income = income[['GEOID', 'median_income', 'margin']]

## combine with OD data

In [113]:
OD_PATH = 'lodes_2021-01-05_transit_drive2'
od = pd.read_csv(f'/home/rishav/Programs/routing/outputs/{OD_PATH}.csv', index_col=0)
od['h_geocode'] = od['h_geocode'].astype(str)
od['w_geocode'] = od['w_geocode'].astype(str)

/tmp/ipykernel_95766/490244206.py:2: DtypeWarning: Columns (32) have mixed types. Specify dtype option on import or set low_memory=False.
  od = pd.read_csv(f'/home/rishav/Programs/routing/outputs/{OD_PATH}.csv', index_col=0)


In [137]:
merged = od.merge(income, left_on='h_geocode', right_on='GEOID')
merged = merged[merged['transit_time'] > merged['drive_time']]
merged = merged[merged['walk_time'] > merged['drive_time']]
merged = merged[merged['walk_time'] >= merged['transit_time']]

merged.to_csv(f'./outputs/{OD_PATH}_demographics.csv')

In [136]:
merged[(merged['transit_time'] > merged['drive_time']) & (merged['walk_time'] > merged['drive_time']) & (merged['walk_time'] > merged['transit_time'])].shape
merged[(merged['walk_time'] >= merged['transit_time'])].shape

(85941, 37)

In [ ]:
http://localhost:8080/otp/routers/default/plan?fromPlace=35.09853204177831,-85.18603221829812&toPlace=35.05385455080387,-85.18299704965646&mode=CAR&date=10-09-2023&time=10:21:00&maxWalkDistance=750
http://localhost:8080/otp/routers/default/plan?fromPlace=35.09853204177831,-85.18603221829812&toPlace=35.05385455080387,-85.18299704965646&mode=WALK,TRANSIT&date=10-09-2023&time=10:21:00&maxWalkDistance=750

http://localhost:8080/otp/routers/default/plan?fromPlace=35.026049682206924,-85.28455717057061&toPlace=35.05220859755195,85.3179341129915&mode=WALK&date=10-09-2023&time=10:21:00&maxWalkDistance=750